In [32]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk import pos_tag
import string
import numpy as np
from numpy import genfromtxt

In [33]:
import pandas as pd
training_data = pd.read_csv("C:\\Users\\karti\\Downloads\\training_data.csv")
testing_data = pd.read_csv("C:\\Users\\karti\\Downloads\\testing_data.csv")

In [34]:
training_data

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...
10975,569934458364813313,neutral,American,NaN,Cottopanama85,NaN,0,@AmericanAir followback,NaN,2015-02-23 10:58:58 -0800,"ohio,panama",NaN
10976,568564006329434113,positive,United,NaN,PaulBEsteves,NaN,0,@united thanks for the help. Wish the phone re...,NaN,2015-02-19 16:13:17 -0800,Brooklyn,Eastern Time (US & Canada)
10977,569643648910028801,negative,US Airways,NaN,runfixsteve,NaN,0,@usairways the. Worst. Ever. #dca #customerser...,NaN,2015-02-22 15:43:24 -0800,"St. Augustine, Florida",NaN
10978,568864981917110272,negative,US Airways,NaN,CLChicosky,NaN,0,@nrhodes85: look! Another apology. DO NOT FLY ...,NaN,2015-02-20 12:09:15 -0800,NaN,NaN


In [35]:
testing_data

,tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,569682010270101504,American,NaN,zsalim03,NaN,0,@AmericanAir In car gng to DFW. Pulled over 1h...,NaN,2015-02-22 18:15:50 -0800,Texas,Central Time (US & Canada)
1,569608307184242688,American,NaN,sa_craig,NaN,0,"@AmericanAir after all, the plane didn’t land ...",NaN,2015-02-22 13:22:57 -0800,"College Station, TX",Central Time (US & Canada)
2,567879304593408001,Southwest,NaN,DanaChristos,NaN,1,@SouthwestAir can't believe how many paying cu...,NaN,2015-02-17 18:52:31 -0800,CT,Eastern Time (US & Canada)
3,569757651539660801,US Airways,NaN,rossj987,NaN,0,@USAirways I can legitimately say that I would...,NaN,2015-02-22 23:16:24 -0800,"Washington, D.C.",Eastern Time (US & Canada)
4,569900705852608513,American,NaN,tranpham18,NaN,0,@AmericanAir still no response from AA. great ...,NaN,2015-02-23 08:44:51 -0800,New York City,Eastern Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...
3655,570304244001193984,US Airways,NaN,Anthony_Scerri,NaN,0,@USAirways Been stuck for 40+ minutes due to l...,NaN,2015-02-24 11:28:22 -0800,"Astoria, NY",Quito
3656,567847737061941249,US Airways,NaN,mttdprkr,NaN,0,@USAirways 4 hours... 4 hours... FOUR HOURS. ...,NaN,2015-02-17 16:47:05 -0800,"Vancouver, WA",Pacific Time (US & Canada)
3657,567823564167192576,Virgin America,NaN,miaerolinea,NaN,1,Nice RT @VirginAmerica: The man of steel might...,NaN,2015-02-17 15:11:02 -0800,Worldwide,Caracas
3658,570273819287531520,American,NaN,GoldensPleasure,NaN,0,@AmericanAir Aww Thanks AA..DFW was on GMA up ...,NaN,2015-02-24 09:27:28 -0800,East Coast CT.,Central Time (US & Canada)


In [36]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [37]:
#finding the x_train and y_train data from training_data
x_train = training_data['text']
y_train = training_data['airline_sentiment']
print(len(x_train), len(y_train))
word_tokenize(x_train[0])

10980 10980


['@',
 'SouthwestAir',
 'I',
 'am',
 'scheduled',
 'for',
 'the',
 'morning',
 ',',
 '2',
 'days',
 'after',
 'the',
 'fact',
 ',',
 'yes',
 '..',
 'not',
 'sure',
 'why',
 'my',
 'evening',
 'flight',
 'was',
 'the',
 'only',
 'one',
 'Cancelled',
 'Flightled']

In [38]:
#finding the x_test data from testing_data
x_test = testing_data['text']
print(len(x_test))
word_tokenize(x_test[0])

3660


['@',
 'AmericanAir',
 'In',
 'car',
 'gng',
 'to',
 'DFW',
 '.',
 'Pulled',
 'over',
 '1hr',
 'ago',
 '-',
 'very',
 'icy',
 'roads',
 '.',
 'On-hold',
 'with',
 'AA',
 'since',
 '1hr',
 '.',
 'Ca',
 "n't",
 'reach',
 'arpt',
 'for',
 'AA2450',
 '.',
 'Wat',
 '2',
 'do',
 '?']

In [39]:
training_doc = []
for i in range(len(x_train)):
    words = word_tokenize(x_train[i])
    training_doc.append((words, y_train[i]))

In [40]:
training_doc[:5]

[(['@',
   'SouthwestAir',
   'I',
   'am',
   'scheduled',
   'for',
   'the',
   'morning',
   ',',
   '2',
   'days',
   'after',
   'the',
   'fact',
   ',',
   'yes',
   '..',
   'not',
   'sure',
   'why',
   'my',
   'evening',
   'flight',
   'was',
   'the',
   'only',
   'one',
   'Cancelled',
   'Flightled'],
  'negative'),
 (['@',
   'SouthwestAir',
   'seeing',
   'your',
   'workers',
   'time',
   'in',
   'and',
   'time',
   'out',
   'going',
   'above',
   'and',
   'beyond',
   'is',
   'why',
   'I',
   'love',
   'flying',
   'with',
   'you',
   'guys',
   '.',
   'Thank',
   'you',
   '!'],
  'positive'),
 (['@',
   'united',
   'Flew',
   'ORD',
   'to',
   'Miami',
   'and',
   'back',
   'and',
   'had',
   'great',
   'crew',
   ',',
   'service',
   'on',
   'both',
   'legs',
   '.',
   'THANKS'],
  'positive'),
 (['@',
   'SouthwestAir',
   '@',
   'dultch97',
   'that',
   "'s",
   'horse',
   'radish',
   '😤🐴'],
  'negative'),
 (['@',
   'united',
   's

In [41]:
testing_doc = []
for i in range(len(x_test)):
    words = word_tokenize(x_test[i])
    testing_doc.append((words))

In [42]:
testing_doc[:5]

[['@',
  'AmericanAir',
  'In',
  'car',
  'gng',
  'to',
  'DFW',
  '.',
  'Pulled',
  'over',
  '1hr',
  'ago',
  '-',
  'very',
  'icy',
  'roads',
  '.',
  'On-hold',
  'with',
  'AA',
  'since',
  '1hr',
  '.',
  'Ca',
  "n't",
  'reach',
  'arpt',
  'for',
  'AA2450',
  '.',
  'Wat',
  '2',
  'do',
  '?'],
 ['@',
  'AmericanAir',
  'after',
  'all',
  ',',
  'the',
  'plane',
  'didn',
  '’',
  't',
  'land',
  'in',
  'identical',
  'or',
  'worse',
  ')',
  'conditions',
  'at',
  'GRK',
  'according',
  'to',
  'METARs',
  '.'],
 ['@',
  'SouthwestAir',
  'ca',
  "n't",
  'believe',
  'how',
  'many',
  'paying',
  'customers',
  'you',
  'left',
  'high',
  'and',
  'dry',
  'with',
  'no',
  'reason',
  'for',
  'flight',
  'Cancelled',
  'Flightlations',
  'Monday',
  'out',
  'of',
  'BDL',
  '!',
  'Wow',
  '.'],
 ['@',
  'USAirways',
  'I',
  'can',
  'legitimately',
  'say',
  'that',
  'I',
  'would',
  'have',
  'rather',
  'driven',
  'cross',
  'country',
  'than',


In [43]:
lemmmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
stop_words.update(list(string.punctuation))

In [44]:
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [45]:
def clean_review(words):
    output_words = []
    for word in words:
        if word.lower() not in stop_words:
            pos = pos_tag([word])
            clean_word = lemmmatizer.lemmatize(word, pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_word)
    return output_words

In [46]:
clean_training_doc = [(clean_review(word_list), category) for word_list, category in training_doc]
clean_testing_doc = [(clean_review(word_list)) for word_list in testing_doc]

In [48]:
clean_training_doc[0]

(['SouthwestAir',
  'schedule',
  'morning',
  '2',
  'day',
  'fact',
  'yes',
  '..',
  'sure',
  'even',
  'flight',
  'one',
  'Cancelled',
  'Flightled'],
 'negative')

In [49]:
clean_testing_doc[0]

['AmericanAir',
 'car',
 'gng',
 'DFW',
 'Pulled',
 '1hr',
 'ago',
 'icy',
 'road',
 'On-hold',
 'AA',
 'since',
 '1hr',
 'Ca',
 "n't",
 'reach',
 'arpt',
 'AA2450',
 'Wat',
 '2']

COUNT VECTORIZER 

In [50]:
from sklearn.feature_extraction.text import CountVectorizer

In [51]:
y_train = [category for doc, category in clean_training_doc]
x_train = [" ".join(doc) for doc, category in clean_training_doc]

In [52]:
x_test = [" ".join(doc) for doc in clean_testing_doc]

In [53]:
x_train[0]

'SouthwestAir schedule morning 2 day fact yes .. sure even flight one Cancelled Flightled'

In [54]:
x_test[0]

"AmericanAir car gng DFW Pulled 1hr ago icy road On-hold AA since 1hr Ca n't reach arpt AA2450 Wat 2"

EXPERIMENT WITH NGRAMS AND TF-IDF

In [55]:
count_vec = CountVectorizer(max_features = 5000)

In [56]:
a = count_vec.fit_transform(x_train)
x_train_features = a.todense()

In [57]:
count_vec.get_feature_names()

['00',
 '000',
 '0016',
 '00pm',
 '02',
 '03',
 '05',
 '05am',
 '05pm',
 '08',
 '10',
 '100',
 '1000',
 '1007',
 '101',
 '1027',
 '103',
 '1041',
 '105',
 '1051',
 '1071',
 '1080',
 '1098',
 '10a',
 '10am',
 '10hrs',
 '10mins',
 '10pm',
 '10th',
 '11',
 '117',
 '1171',
 '118',
 '11a',
 '11am',
 '11pm',
 '11th',
 '12',
 '120',
 '1200',
 '1230',
 '1242',
 '1254',
 '12am',
 '12b',
 '13',
 '130',
 '1357',
 '1359',
 '136',
 '1388',
 '1389',
 '13th',
 '14',
 '140',
 '1472',
 '1491',
 '15',
 '150',
 '1500',
 '1503',
 '1514',
 '152',
 '1531',
 '1533',
 '1535',
 '1547',
 '1562',
 '1583',
 '15minutes',
 '15th',
 '16',
 '1625',
 '1644',
 '1657',
 '1679',
 '17',
 '1700',
 '1701',
 '1706',
 '1715',
 '174',
 '1750',
 '17th',
 '18',
 '180',
 '1800',
 '1808',
 '1826',
 '1861',
 '188',
 '1898',
 '19',
 '190',
 '1917',
 '195',
 '1970',
 '1971',
 '1997',
 '1am',
 '1hr',
 '1k',
 '1pm',
 '1st',
 '1way',
 '20',
 '200',
 '2000',
 '2012',
 '2014',
 '2015',
 '2016',
 '2034',
 '20l',
 '20min',
 '20pm',
 '21',
 

In [58]:
b = count_vec.transform(x_test)
x_test_features = b.todense()

APPLY SKLEARN CLASSIFIER

In [59]:
from sklearn.svm import SVC

In [60]:
svc = SVC()

In [61]:
svc.fit(x_train_features, y_train)

SVC()

In [62]:
y_pred = svc.predict(x_test_features)

In [64]:
y_pred

array(['negative', 'negative', 'negative', ..., 'neutral', 'positive',
       'negative'], dtype='<U8')

SAVING THE PREDICTIONS TO CSV FILE

In [68]:
len(x_test_features), len(y_pred)

(3660, 3660)

In [71]:
y_pred[-1]

'negative'

In [72]:
import csv
file = open("twitter_sentiment_output.csv", "w")
wtr = csv.writer(file, delimiter=',', lineterminator='\n')
count = 0
for x in y_pred : 
    count += 1

    wtr.writerow ([x])
count

3660